In [4]:
# encoding=utf-8
import re
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [5]:
print('load data begin ...')

df_raw = pd.read_csv("../data/train_data_nlp_and_number_and_category_feature_multi_label_large.txt", header=0, error_bad_lines=False, delimiter='\t', engine='python',na_values="");

print('df_raw.shape=',df_raw.shape)
print('load data end ...')

load data begin ...


/tmp/ipykernel_968128/1183947550.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_raw = pd.read_csv("../data/train_data_nlp_and_number_and_category_feature_multi_label_large.txt", header=0, error_bad_lines=False, delimiter='\t', engine='python',na_values="");


df_raw.shape= (874006, 263)
load data end ...


In [6]:
#df_raw.describe()
df_raw.head(10)

,user_id,c_label,r_label,problem_content,main_belong_loc1,main_belong_cate2,main_city_hierarchy,free_money_avg_1d,free_money_avg_3d,free_money_avg_7d,...,pc_login_counts_7d,pc_login_counts_14d,pc_login_counts_30d,pc_login_counts_60d,m_login_counts_1d,m_login_counts_3d,m_login_counts_7d,m_login_counts_14d,m_login_counts_30d,m_login_counts_60d
0,12396036,2,2,采购员客服发布招聘信息刷新优化#电话联系 设置采购 客服精品刷新 效果不好 建议客户多发帖#...,7578,3112,二线,0.000000,0.002022,0.006067,...,7,14,30,53,0,0,0,0,0,0
1,13225122,2,2,15666066288#15666066288#15666066288,372,706,二线,0.000000,0.149213,0.447640,...,0,0,0,2,0,0,0,0,0,0
2,13859470,1,2,NaN,8719,3133,四线,12.104182,36.912000,90.032000,...,7,14,30,51,0,0,0,0,0,0
3,16386667,0,1,对接人宋老板，电话联系客户拒接下次回访时间1219#对接人宋老板，电话联系客户表示没有招聘需...,623,622,四线,0.506222,13.407000,55.430556,...,4,5,5,5,0,0,0,0,0,0
4,16449418,2,2,应客户要求，送货员要求精品展示,6921,580,四线,0.000000,0.000000,0.000000,...,0,0,0,5,0,0,0,0,0,0
5,16462316,2,2,帮客户手动刷新招聘信息 提醒客户多去使用资源 #15673298100提醒客户有时间的时候可...,2047,620,四线,38.666667,116.000000,270.666667,...,0,1,2,3,0,0,0,0,0,0
6,16462316,0,2,NaN,2047,620,四线,37.866667,115.200000,269.866667,...,1,1,1,2,0,0,0,0,0,0
7,17114931,0,2,不招聘#年前不再招聘，暂停所有信息#电话沟通客户，年前不再招聘，暂停所有信息,362,580,二线,4.291222,12.873667,30.038556,...,4,4,4,4,0,0,0,0,0,0
8,17965537,2,2,18301622122帮客户发布招聘信息，跟进人招聘客服王惠珂#18301622122帮客户...,882,625,四线,0.000000,69.246292,207.738876,...,5,12,28,37,0,0,0,0,0,0
9,18034222,2,2,没有需要,5722,607,五线,0.000000,0.000000,0.000000,...,7,14,29,59,0,0,0,0,0,0


In [7]:
print(df_raw.dtypes)
str_values = {"problem_content": "", "main_city_hierarchy": ""}
df_raw.fillna(value=str_values,inplace=True) ##去除Nan，空字符串也被看作是nan，因为读文件的时候设定了na_values=""
df_data=df_raw
df_data.head()

user_id                int64
c_label                int64
r_label                int64
problem_content       object
main_belong_loc1       int64
                       ...  
m_login_counts_3d      int64
m_login_counts_7d      int64
m_login_counts_14d     int64
m_login_counts_30d     int64
m_login_counts_60d     int64
Length: 263, dtype: object


,user_id,c_label,r_label,problem_content,main_belong_loc1,main_belong_cate2,main_city_hierarchy,free_money_avg_1d,free_money_avg_3d,free_money_avg_7d,...,pc_login_counts_7d,pc_login_counts_14d,pc_login_counts_30d,pc_login_counts_60d,m_login_counts_1d,m_login_counts_3d,m_login_counts_7d,m_login_counts_14d,m_login_counts_30d,m_login_counts_60d
0,12396036,2,2,采购员客服发布招聘信息刷新优化#电话联系 设置采购 客服精品刷新 效果不好 建议客户多发帖#...,7578,3112,二线,0.000000,0.002022,0.006067,...,7,14,30,53,0,0,0,0,0,0
1,13225122,2,2,15666066288#15666066288#15666066288,372,706,二线,0.000000,0.149213,0.447640,...,0,0,0,2,0,0,0,0,0,0
2,13859470,1,2,,8719,3133,四线,12.104182,36.912000,90.032000,...,7,14,30,51,0,0,0,0,0,0
3,16386667,0,1,对接人宋老板，电话联系客户拒接下次回访时间1219#对接人宋老板，电话联系客户表示没有招聘需...,623,622,四线,0.506222,13.407000,55.430556,...,4,5,5,5,0,0,0,0,0,0
4,16449418,2,2,应客户要求，送货员要求精品展示,6921,580,四线,0.000000,0.000000,0.000000,...,0,0,0,5,0,0,0,0,0,0


In [8]:
#https://github.com/oussamaErra/tf-2-tutorial-categorical-features-embedding/blob/master/cat-features-embeddings-model.py
from sklearn.preprocessing import LabelEncoder
category_cols=["main_belong_loc1","main_belong_cate2","main_city_hierarchy"]
for col in category_cols:
    encoder = LabelEncoder()
    df_data[col] = encoder.fit_transform(df_data[col])
print(df_data[category_cols].head())
print("category feature process end")

   main_belong_loc1  main_belong_cate2  main_city_hierarchy
0               444                130                    2
1                48                 89                    2
2               466                138                    5
3               111                 79                    5
4               411                 58                    5
category feature process end


In [16]:
df_data["main_city_hierarchy"].sort_values().unique()
df_data["main_belong_loc1"].sort_values().unique()
#df_data["main_belong_cate2"].sort_values().unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
inputs_category = []
embed_dim_category=10
model_block_category=[]
for col in category_cols:
    vocab_size = df_data[col].nunique()
    inpt = tf.keras.layers.Input(shape=(1,),name='input_'+col)
    inputs_category.append(inpt)
    embed = tf.keras.layers.Embedding(vocab_size,embed_dim_category, trainable=True,embeddings_initializer=tf.initializers.random_normal)(inpt)
    #print(embed.shape)
    embed_rehsaped =tf.keras.layers.Reshape(target_shape=(embed_dim_category,))(embed)
    print(embed_rehsaped.shape)
    model_block_category.append(embed_rehsaped)
    

In [ ]:

full_text=''
print(len(df_data.index))
for i in range(df_data.shape[0]):
    text=df_data.iloc[i,3]
    text=re.sub('[，。；？（）！、【】!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ]','',text)    
    full_text+=text

print("full_text process end")

In [ ]:

tokenizer = Tokenizer(num_words=2000,
                      #filters='，。；？（）！-【】!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', 
                      lower=False, 
                      split='', 
                      char_level=True, #if set True,filters will has no effect
                      oov_token='<OOV>', 
                      document_count=0
)
tokenizer.fit_on_texts(full_text)
tokenizer.word_counts
tokenizer.word_index


In [ ]:
max_len=100
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
problem_content_list=[]
X_number_feature=[]
X_category_feature=[]

for i in range(df_data.shape[0]):
    c_label=df_data.iloc[i,1]
    r_label=df_data.iloc[i,2]
    problem_content_list.append(re.sub('[，。；？（）！【】!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ]','',df_data.iloc[i,3])) ##注意此处必须是list形式
    X_category_feature.append(df_data.iloc[i,4:7].values)
    X_number_feature.append(df_data.iloc[i,7:].values)
       
sequence_indx_list=tokenizer.texts_to_sequences(problem_content_list)
sequence_indx_pad_list=sequence.pad_sequences(sequence_indx_list,padding="post",maxlen=max_len,truncating='post',)


#print(sequence_indx_list)
print(sequence_indx_pad_list)


In [ ]:
##转化成numpy的ndarray
X_number_feature=np.asarray(X_number_feature)
X_category_feature=np.asarray(X_category_feature)
X_text_feature=sequence_indx_pad_list

print("text feature----")
print("vocab max=",X_text_feature.max())
print(X_text_feature.shape)
print(X_text_feature[0])
print("category feature----")
print(X_category_feature.shape)
print(X_category_feature[0])
print("number feature----")
print(X_number_feature.shape)
print(X_number_feature[0])


In [ ]:
X=np.concatenate((X_text_feature,X_category_feature,X_number_feature),axis=1)
print(X.shape)
print(len(X[0]))
print(X[0])

In [ ]:
c_Y=df_data['c_label'].values.tolist()
r_Y=df_data['r_label'].values.tolist()
#zipped_label= list(zip(c_Y, r_Y)) python2中使用list(zip(list1,list2))
zipped_label=[*zip(c_Y,r_Y)]
Y=np.array(zipped_label)
print(Y[0:10])
print(Y[:,0])


In [ ]:
print(X.shape)
print(Y.shape)
print(len(c_Y))
print(len(r_Y))


In [ ]:
#x_train_val, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=299)

x_train_valid, x_test, y_train_valid, y_test = train_test_split(X, Y, test_size=0.1, random_state=299)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.1, random_state=399)

print('x_train.shape=', x_train.shape)
print('y_train.shape=', y_train.shape)

print('x_valid.shape=', x_valid.shape)
print('y_valid.shape=', y_valid.shape)

print('x_test.shape=', x_test.shape)
print('y_test.shape=', y_test.shape)



In [ ]:
from collections import Counter
counter_cost_train=Counter(y_train[:,0])
counter_recharge_train=Counter(y_train[:,1])
counter_cost_test=Counter(y_test[:,0])
counter_recharge_test=Counter(y_test[:,1])

print("counter_cost_train=",counter_cost_train)
print("counter_recharge_train=",counter_recharge_train)
print("counter_cost_test=",counter_cost_test)
print("counter_recharge_test=",counter_recharge_test)

In [ ]:
#text tokenize feature 100个
X1_train=x_train[:,0:max_len]
X1_test=x_test[:,0:max_len]
X1_valid=x_valid[:,0:max_len]

# category feature 3个
X3_train=x_train[:,max_len:max_len+3]
X3_test=x_test[:,max_len:max_len+3]
X3_valid=x_valid[:,max_len:max_len+3]

# number feature
X2_train=x_train[:,max_len+3:]
X2_test=x_test[:,max_len+3:]
X2_valid=x_valid[:,max_len+3:]





X1_train=np.asarray(X1_train).astype(np.int32) ###此处加上这个是为了解决：Failed to convert a NumPy array to a Tensor (Unsupported object type int).
X1_test=np.asarray(X1_test).astype(np.int32)
X1_valid=np.asarray(X1_valid).astype(np.int32)

X2_train=np.asarray(X2_train).astype(np.int32)
X2_test=np.asarray(X2_test).astype(np.int32)
X2_valid=np.asarray(X2_valid).astype(np.int32)

X3_train=np.asarray(X3_train).astype(np.int32)
X3_test=np.asarray(X3_test).astype(np.int32)
X3_valid=np.asarray(X3_valid).astype(np.int32)

print('X1_train=',X1_train.shape)
print('X1_test=',X1_test.shape)
print('X1_valid=',X1_valid.shape)
print('X2_train=',X2_train.shape)
print('X2_test=',X2_test.shape)
print('X2_valid=',X2_valid.shape)
print('X3_train=',X3_train.shape)
print('X3_test=',X3_test.shape)
print('X3_valid=',X3_valid.shape)




In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Embedding, Dense, LSTM, Bidirectional,Dropout
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy#,CategoricalCrossentropy
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Normalization


# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 64
#loss_function = BinaryCrossentropy()
#loss_function = CategoricalCrossentropy
loss_function = SparseCategoricalCrossentropy(from_logits=True) # set True in order to solve "Expected float32, got 'auto' of type 'str' instead."
max_sequence_length = max_len

#num_distinct_words 此处是num_words=2000还是vocab_size？"max=",sequence_indx_pad_list.max()=1999，2000=1+1999
num_distinct_words = 2000 

number_of_epochs = 100
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1


In [ ]:
normalization_layer = Normalization(name='normalization',axis=-1)
normalization_layer.adapt(X_number_feature.astype(np.float32))
print("normalization end")

In [ ]:
print(normalization_layer.adapt_mean.shape)

In [ ]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz


def create_multi_task_learning_model():
    input1=Input(shape=(max_sequence_length,),dtype=tf.int32,name='input_text')
    x1=Embedding(input_dim=num_distinct_words, output_dim=embedding_output_dims, input_length=max_sequence_length,name='Embedding_text')(input1)
    x1=Bidirectional(LSTM(20,activation="relu",dropout=0.2,return_sequences=True),name='BiLstm_text1')(x1)
    x1=Bidirectional(LSTM(20,activation="relu",dropout=0.2,), merge_mode='sum',name='BiLstm_text2')(x1)
    x1=Dense(10, activation='relu',name='Dense_text1')(x1)
    x1=Dropout(0.3,name='dropout_text1')(x1)

    #output1=Dense(3, activation='softmax',name='Dense_text2')(x1)

    input2=Input(shape=(X2_train.shape[1],),dtype=tf.float32,name='input_number')
    x2=normalization_layer(input2)
    x2=Dense(40, activation='relu',name='Dense_number1')(x2)
    x2=Dropout(0.3)(x2)

    x3=concatenate([x1,x2]+model_block_category)

    x3_1=Dense(20, activation='relu',name='Dense_cost_1')(x3)
    x3_1=Dropout(0.3)(x3_1)
    output_cost=Dense(3, activation='softmax',name='dense_output_cost')(x3_1)

    x3_2=Dense(20, activation='relu',name='Dense_recharge_1')(x3)
    x3_2=Dropout(0.3)(x3_2)
    output_recharge=Dense(3, activation='softmax',name='dense_output_recharge')(x3_2)
    model = Model(inputs=[input1,input2]+inputs_category, outputs=[output_cost,output_recharge])
    
    #model.summary()
    #tf.keras.utils.plot_model(model, to_file='model.png')

    return model
    

In [ ]:
def compile_multitask_model(model, gamma):
        
    model.compile(optimizer='adam',
                  loss={'dense_output_cost': 'sparse_categorical_crossentropy', 
                        'dense_output_recharge': 'sparse_categorical_crossentropy'},
                  loss_weights={'dense_output_cost': gamma, 
                                'dense_output_recharge': 1 - gamma}, 
                  metrics=['accuracy'])
        
    return model

In [ ]:
import time


def fit_batch(gamma_values):
    
    history = list()
    trained_models = list()
    
    print('Starting training on batch of models for gamma values ', gamma_values, '\n\n')
    
    for gamma in gamma_values:
        
        print('Training model for gamma equal to ', gamma)
        model = create_multi_task_learning_model()
        
        model = compile_multitask_model(model, gamma)
        
        model.summary()
        
        tf.keras.utils.plot_model(model, to_file='./model_1.png', show_shapes=True)

        
        start = time.time()
        model_history =model.fit(x={'input_text': X1_train,
                                  'input_number': X2_train,
                                  'input_main_belong_loc1':X3_train[:,0],
                                  'input_main_belong_cate2':X3_train[:,1],
                                  'input_main_city_hierarchy':X3_train[:,2]
                                 },
                                 y={'dense_output_cost': y_train[:,0], 'dense_output_recharge': y_train[:,1]},
                                 
                                 validation_data=({'input_text': X1_valid,
                                  'input_number': X2_valid,
                                  'input_main_belong_loc1':X3_valid[:,0],
                                  'input_main_belong_cate2':X3_valid[:,1],
                                  'input_main_city_hierarchy':X3_valid[:,2]
                                 },
                                  {'dense_output_cost': y_valid[:,0], 'dense_output_recharge': y_valid[:,1]}                
                                ),
                            epochs=10, batch_size=128, verbose=1)
        print(f'Training time: {time.time() - start}\n')
        history.append(model_history)
        trained_models.append(model)
        
    return history, trained_models

In [ ]:
training_history, trained_models = fit_batch([0.6])

In [ ]:
import matplotlib.pyplot as plt


def plot_multitask_accuracies(gammas, training_history):
    
    counter = 0

    for history in training_history:

        print(f'\nPlotting Accuracy vs Epochs for value of gamma number {gammas[counter]}\n')
        plt.plot(range(len(history.history['dense_output_cost_accuracy'])), history.history['dense_output_cost_accuracy'], c='r', label='Cost')
        plt.plot(range(len(history.history['dense_output_recharge_accuracy'])), history.history['dense_output_recharge_accuracy'], c='b', label='Recharge')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()
        counter += 1

In [ ]:
# Test the model after training
model = trained_models[0]
test_results = model.evaluate(x={'input_text': X1_test,
                                  'input_number': X2_test,
                                  'input_main_belong_loc1':X3_test[:,0],
                                  'input_main_belong_cate2':X3_test[:,1],
                                  'input_main_city_hierarchy':X3_test[:,2]
                                 },
                                 y={'dense_output_cost': y_test[:,0], 'dense_output_recharge': y_test[:,1]}, batch_size=128,verbose=True)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

In [ ]:
#for pred in model.predict(x=[X1_train,X2_train],verbose=1,use_multiprocessing=True):
#    print(pred.shape)
    #print('Task prediction: ', np.argmax(pred))

predict_result=model.predict(x={'input_text': X1_test,
                                  'input_number': X2_test,
                                  'input_main_belong_loc1':X3_test[:,0],
                                  'input_main_belong_cate2':X3_test[:,1],
                                  'input_main_city_hierarchy':X3_test[:,2]
                                 },verbose=1,use_multiprocessing=True)

In [ ]:
predict_result=np.asarray(predict_result)
print(predict_result.shape)
print(predict_result[0][0])
print("\n")
print(predict_result[1][0])

In [ ]:
predict_prob_cost=predict_result[0]
predict_prob_recharge=predict_result[1]

In [ ]:
## this is a test
print(type(predict_prob_cost))
print(predict_prob_cost.shape)
print(predict_prob_cost[0])
print(predict_prob_cost)
predit_label_cost = np.asarray([np.argmax(v) for v in predict_prob_cost])
print(predit_label_cost.shape)

print("predict prob= \n",predict_prob_cost[0:2])
print("predict label= \n",predit_label_cost[0:2])

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def evaluate_score(y_true,y_pred_prob):
    pred_label=np.asarray([np.argmax(v) for v in y_pred_prob])
    print('accuracy: %.3f' % accuracy_score(y_true, pred_label))
    print('ovr macro auc: %.3f' % roc_auc_score(y_true, y_pred_prob,multi_class='ovo',average='macro')) #注意这里是prob
    print('ovo macro auc: %.3f' % roc_auc_score(y_true, y_pred_prob, multi_class='ovr',average='macro')) #注意这里是prob
    print('ovr weighted auc: %.3f' % roc_auc_score(y_true, y_pred_prob,multi_class='ovo',average='weighted')) #注意这里是prob
    print('ovo weighted auc: %.3f' % roc_auc_score(y_true, y_pred_prob, multi_class='ovr',average='weighted')) #注意这里是prob
    print('micro precision: %.3f' % precision_score(y_true, pred_label,average='micro'))
    print('macro precision: %.3f' % precision_score(y_true, pred_label,average='macro'))
    print('weighted precision: %.3f' % precision_score(y_true, pred_label,average='weighted'))
    print('micro recall: %.3f' % recall_score(y_true, pred_label,average='micro'))
    print('macro recall: %.3f' % recall_score(y_true, pred_label,average='macro'))
    print('weighted recall: %.3f' % recall_score(y_true, pred_label,average='weighted'))
    

In [ ]:
print("-----------消耗模型评估分值: ---------")
evaluate_score(y_test[:,0],predict_prob_cost)
print("-----------充值模型评估分值: ---------")
evaluate_score(y_test[:,1],predict_prob_recharge)